In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [4]:
app_train =  pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
# 他社債務
hc_br_train = pd.read_csv('./home-credit-default-risk/exports/hc_br_train.csv')
hc_br_test = pd.read_csv('./home-credit-default-risk/exports/hc_br_test.csv')
# hc Cash loans
hc_only_cash_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_cash_train.csv')
hc_only_cash_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_cash_test.csv')
# hc Revolbing loans
hc_only_revo_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_revo_train.csv')
hc_only_revo_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_revo_test.csv')
## hc Consumer loans
hc_only_consumer_train = pd.read_csv('./home-credit-default-risk/exports/hc_only_consumer_train.csv')
hc_only_consumer_test = pd.read_csv('./home-credit-default-risk/exports/hc_only_consumer_test.csv')

In [5]:
columns = ['TARGET']
hc_br_train.drop(columns, axis=1, inplace=True)
hc_only_cash_train.drop(columns, axis=1, inplace=True)
hc_only_revo_train.drop(columns, axis=1, inplace=True)
hc_only_consumer_train.drop(columns, axis=1, inplace=True)

In [6]:
# trainへマージ
app_train = pd.merge(app_train, hc_br_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_br_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [7]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_cash_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_cash_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [8]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_revo_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_revo_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [9]:
# trainへマージ
app_train = pd.merge(app_train, hc_only_consumer_train, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, hc_only_consumer_test, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [10]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,consumer_amt_application,consumer_amt_credit,consumer_amt_goods_price,consumer_weekday_appr_process_start,consumer_hour_appr_process_start,consumer_name_client_type,consumer_name_goods_category,consumer_name_seller_industry,consumer_days_last_due,consumer_installment_payment_ratio_1000_mean_mean
0,100002,0,1,0,0,202500.0,406597.5,24700.5,351000.0,0,...,179055.0,179055.0,179055.0,0.0,9.0,0.0,0.0,0.0,-25.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,0,0,135000.0,312682.5,29686.5,297000.0,0,...,334917.0,267930.0,334917.0,1.0,15.0,1.0,1.0,1.0,365243.0,0.0
4,100007,0,1,0,0,121500.0,513000.0,21865.5,513000.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    ,'num_round': 10000
    ,'early_stopping_round': 50
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[21:42:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "early_stopping_round", "num_round", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.72453	eval-auc:0.71042
[1]	train-auc:0.74597	eval-auc:0.72453
[2]	train-auc:0.75179	eval-auc:0.73136
[3]	train-auc:0.75471	eval-auc:0.73141
[4]	train-auc:0.75938	eval-auc:0.73468
[5]	train-auc:0.76189	eval-auc:0.73586
[6]	train-auc:0.76420	eval-auc:0.73828
[7]	train-auc:0.76679	eval-auc:0.73976
[8]	train-auc:0.76920	eval-auc:0.74074
[9]	train-auc:0.77040	eval-auc:0.74171
[10]	train-auc:0.77222	eval-auc:0.74252
[11]	train-auc:0.77558	eval-auc:0.74417
[12]	train-auc:0.77715	eval-auc:0.74465
[13]	t

[8]	train-auc:0.76851	eval-auc:0.74134
[9]	train-auc:0.77099	eval-auc:0.74238
[10]	train-auc:0.77325	eval-auc:0.74283
[11]	train-auc:0.77536	eval-auc:0.74267
[12]	train-auc:0.77724	eval-auc:0.74361
[13]	train-auc:0.77998	eval-auc:0.74489
[14]	train-auc:0.78089	eval-auc:0.74554
[15]	train-auc:0.78255	eval-auc:0.74602
[16]	train-auc:0.78412	eval-auc:0.74642
[17]	train-auc:0.78478	eval-auc:0.74666
[18]	train-auc:0.78643	eval-auc:0.74703
[19]	train-auc:0.78864	eval-auc:0.74819
[20]	train-auc:0.78997	eval-auc:0.74917
[21]	train-auc:0.79170	eval-auc:0.75002
[22]	train-auc:0.79314	eval-auc:0.75030
[23]	train-auc:0.79423	eval-auc:0.75034
[24]	train-auc:0.79532	eval-auc:0.75071
[25]	train-auc:0.79679	eval-auc:0.75122
[26]	train-auc:0.79902	eval-auc:0.75236
[27]	train-auc:0.80056	eval-auc:0.75291
[28]	train-auc:0.80200	eval-auc:0.75356
[29]	train-auc:0.80365	eval-auc:0.75374
[30]	train-auc:0.80512	eval-auc:0.75408
[31]	train-auc:0.80672	eval-auc:0.75544
[32]	train-auc:0.80840	eval-auc:0.75578
[3

In [12]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [13]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [14]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [15]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [16]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [17]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [18]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.071213,0.063068,0.069864,0.118157,0.107589
1,0.087155,0.081244,0.089413,0.082116,0.085264
2,0.041302,0.040944,0.041497,0.039541,0.041668
3,0.060204,0.050327,0.056014,0.046020,0.059237
4,0.149999,0.138296,0.124070,0.121593,0.154282


In [19]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.921938,0.927508,0.921358,0.917724
submit_all_01,0.921938,1.000000,0.923488,0.921156,0.922187
submit_all_02,0.927508,0.923488,1.000000,0.922424,0.919430
submit_all_03,0.921358,0.921156,0.922424,1.000000,0.921863
submit_all_04,0.917724,0.922187,0.919430,0.921863,1.000000


In [20]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.085978
1,100005,0.085038
2,100013,0.040990
3,100028,0.054361
4,100038,0.137648


In [21]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_all.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.017196
1,100005,0.017008
2,100013,0.008198
3,100028,0.010872
4,100038,0.027530


In [22]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64